In [ ]:
import pydicom
import numpy as np
import os
from pydicom.pixel_data_handlers.util import convert_color_space

def dicom_to_matrix_recursive(dicom_folder_path, output_folder_path):
    """
    Recursively converts all DICOM files in a specified folder (and its subfolders) to numpy matrices and saves them as .npy files.

    Parameters:
    dicom_folder_path (str): Path to the folder containing DICOM files.
    output_folder_path (str): Path to the folder where .npy files will be saved.
    """

    for root, dirs, files in os.walk(dicom_folder_path):
        for filename in files:
            if filename.endswith(".dcm"):  # Check if the file is a DICOM file
                dicom_path = os.path.join(root, filename)

                try:
                
                    dicom_data = pydicom.dcmread(dicom_path)

                    if dicom_data.file_meta.TransferSyntaxUID.is_compressed:
                        dicom_data.decompress()

                    pixel_array = dicom_data.pixel_array

                    pixel_matrix = np.array(pixel_array)

                    relative_path = os.path.relpath(root, dicom_folder_path)
                    output_dir = os.path.join(output_folder_path, relative_path)
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)

                    output_file_path = os.path.join(output_dir, filename.replace(".dcm", ".npy"))
                    np.save(output_file_path, pixel_matrix)

                    print(f"Converted {dicom_path} to {output_file_path}")

                except Exception as e:
                    print(f"Error processing {dicom_path}: {e}")

dicom_folder_path = 'C:/Users/BUSRA/Downloads/teknofest/demo_dataset'
output_folder_path = 'C:/Users/BUSRA/Documents/Teknofest-2024/npyverilerFİNAL'
dicom_to_matrix_recursive(dicom_folder_path, output_folder_path)


In [29]:
import cv2
import numpy as np
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse

def gaussian_bulaniklastirma(image, kernel_boyutu=(5, 5)):
    return cv2.GaussianBlur(image, kernel_boyutu, 0)

def clahe_uygula(image, clip_limit=1.0, tile_grid_size=(8, 8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    return clahe.apply(image)

def normalize_et(image):
    normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    return normalized_image.astype(np.uint8)

def goruntu_kalitesini_degerlendir(orijinal, islenmis):
    psnr_degeri = psnr(orijinal, islenmis)
    ssim_degeri = ssim(orijinal, islenmis)
    mse_degeri = mse(orijinal, islenmis)
    return psnr_degeri, ssim_degeri, mse_degeri

def roi_cikar(image):
    norm_image = normalize_et(image)
    thresholded_image = cv2.adaptiveThreshold(norm_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                              cv2.THRESH_BINARY_INV, 11, 2)
    kernel = np.ones((5, 5), np.uint8)
    temizlenmis_goruntu = cv2.morphologyEx(thresholded_image, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(temizlenmis_goruntu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        en_buyuk_kontur = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(en_buyuk_kontur)
        roi_image = image[y:y+h, x:x+w]
        return roi_image
    else:
        return image

npy_directory = 'C:/Users/BUSRA/Documents/Teknofest-2024/npyverilerFİNAL'

ana_cikti_klasoru = 'C:/Users/BUSRA/Documents/Teknofest-2024/Islenmis_GoruntulerFİNAL'

psnr_listesi = []
ssim_listesi = []
mse_listesi = []

for root, dirs, files in os.walk(npy_directory):
    for file in files:
        if file.endswith('.npy'):
            dosya_yolu = os.path.join(root, file)
            
            try:
                orijinal_goruntu = np.load(dosya_yolu)

                # Extract ROI
                roi_goruntu = roi_cikar(orijinal_goruntu)

                # Normalize the ROI
                roi_goruntu = normalize_et(roi_goruntu)

                # Noise Reduction: Gaussian Blur
                bulanık_goruntu = gaussian_bulaniklastirma(roi_goruntu)

                # Contrast Enhancement: CLAHE
                clahe_goruntu = clahe_uygula(bulanık_goruntu, clip_limit=1.0, tile_grid_size=(8, 8))

                # Normalization: After CLAHE
                son_goruntu = normalize_et(clahe_goruntu)

                psnr_degeri, ssim_degeri, mse_degeri = goruntu_kalitesini_degerlendir(roi_goruntu, son_goruntu)
                psnr_listesi.append(psnr_degeri)
                ssim_listesi.append(ssim_degeri)
                mse_listesi.append(mse_degeri)

                yeni_dosya_yolu = dosya_yolu.replace('npyveriler', 'Islenmis_Goruntuler').replace('.npy', '_processed.npy')
                os.makedirs(os.path.dirname(yeni_dosya_yolu), exist_ok=True)

                np.save(yeni_dosya_yolu, son_goruntu)

            except Exception as e:
                print(f"{dosya_yolu} dosyası işlenirken hata oluştu: {e}")

# Calculate and print average metric values
ortalama_psnr = np.mean(psnr_listesi)
ortalama_ssim = np.mean(ssim_listesi)
ortalama_mse = np.mean(mse_listesi)

print(f"Ortalama PSNR: {ortalama_psnr:.2f}")
print(f"Ortalama SSIM: {ortalama_ssim:.4f}")
print(f"Ortalama MSE: {ortalama_mse:.2f}")


Ortalama PSNR: 26.45
Ortalama SSIM: 0.8833
Ortalama MSE: 172.60


In [ ]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt

# İşlenmiş görüntülerin bulunduğu dizin
islenmis_goruntu_klasoru = 'C:/Users/BUSRA/Documents/Teknofest-2024/Islenmis_Goruntuler'

# Görüntüleme yapılacak görüntü sayısı
gosterilecek_goruntu_sayisi = 20

# Tüm .npy dosyalarını bulma
islenmis_dosyalar = []
for root, dirs, files in os.walk(islenmis_goruntu_klasoru):
    for file in files:
        if file.endswith('_processed.npy'):
            islenmis_dosyalar.append(os.path.join(root, file))

# Eğer dosya yoksa çıkış yap
if not islenmis_dosyalar:
    print("İşlenmiş dosya bulunamadı.")
else:
    # Rastgele dosyaları seçme
    secilen_dosyalar = random.sample(islenmis_dosyalar, min(gosterilecek_goruntu_sayisi, len(islenmis_dosyalar)))

    # Seçilen dosyaları görüntüleme
    for dosya_yolu in secilen_dosyalar:
        # .npy dosyasını yükle
        goruntu = np.load(dosya_yolu)
        
        # Görüntüyü göster
        plt.figure()
        plt.imshow(goruntu, cmap='gray')
        plt.title(f'Görüntü: {os.path.basename(dosya_yolu)}')
        plt.axis('off')
    
    # Görüntüleri göster
    plt.show()


In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import exposure
import pandas as pd

# CSV dosyasının yolunu belirtin
csv_file_path = 'C:/Users/BUSRA/Documents/Teknofest-2024/Dataframe_NPY.csv'  # Kendi CSV dosyanızın yolunu belirtin

# CSV dosyasını oku
df = pd.read_csv(csv_file_path)

# Custom Dataset Sınıfı
class MammographyDataset(Dataset):
    def __init__(self, dataframe, transform=None, apply_clahe=False):
        self.dataframe = dataframe
        self.transform = transform
        self.apply_clahe = apply_clahe
        self.missing_files = []

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['NPY DOSYA YOLU']
        label = self.dataframe.iloc[idx]['birads']

        label = 1 if label >= 2 else 0

        try:
            img_data = np.load(img_path)
            img_data = Image.fromarray(img_data)
            
            if self.apply_clahe:
                img_data = np.array(img_data)
                img_data = exposure.equalize_adapthist(img_data)  # CLAHE uygulama
                img_data = Image.fromarray((img_data * 255).astype(np.uint8))

            if img_data.mode != 'L':
                img_data = img_data.convert('L')

            if self.transform:
                img_data = self.transform(img_data)

            return img_data, label

        except FileNotFoundError:
            self.missing_files.append(img_path)
            print(f"Dosya bulunamadı: {img_path}")
            return None, None

# Özelleştirilmiş collate_fn fonksiyonu
def custom_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    
    if len(batch) == 0:
        return torch.empty(0), torch.empty(0)
    
    images, labels = zip(*batch)
    return torch.stack(images, 0), torch.tensor(labels)

# Veriyi Eğitim, Doğrulama ve Test Setlerine Bölmek
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Veri Dönüşümleri
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=3),  # ResNet ve AlexNet RGB bekler
    transforms.ToTensor()
])

# Dataset ve DataLoader Oluşturma
train_dataset = MammographyDataset(dataframe=train_df, transform=transform, apply_clahe=True)
valid_dataset = MammographyDataset(dataframe=valid_df, transform=transform)
test_dataset = MammographyDataset(dataframe=test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)

# AlexNet ve ResNet50 Ensemble Modeli
class EnsembleModel(nn.Module):
    def __init__(self):
        super(EnsembleModel, self).__init__()
        
        # ResNet50 modeli
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Son sınıflandırma katmanını kaldır
        
        # AlexNet modeli
        self.alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
        self.alexnet.classifier[6] = nn.Identity()  # Son sınıflandırma katmanını kaldır
        
        self.dnn = nn.Sequential(
            nn.Linear(2048 + 4096, 512),  # Birleştirilmiş özelliklerin giriş boyutunu düzelt
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x1 = self.resnet(x)  
        x2 = self.alexnet(x)  

        x_combined = torch.cat((x1, x2), dim=1)

        x_out = self.dnn(x_combined)
        return x_out

model = EnsembleModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            if images.size(0) == 0:
                continue
            
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()
                model.cuda()
            
            optimizer.zero_grad()
            
            try:
                outputs = model(images)
            except RuntimeError as e:
                print(f"Model forward geçiş hatası: {e}")
                print(f"Girdi boyutları: {images.shape}")
                continue

            loss = criterion(outputs, labels.unsqueeze(1).float())
            
            try:
                loss.backward()
            except RuntimeError as e:
                print(f"Loss backward geçiş hatası: {e}")
                continue

            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')


def validate_model(model, valid_loader):
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for images, labels in valid_loader:
            if images.size(0) == 0:
                continue
            
           
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()
                model.cuda()
            
            outputs = model(images)
            preds = (outputs > 0.5).float()
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(val_labels, val_preds)
    print(f'Validation Accuracy: {accuracy:.4f}')


def test_model(model, test_loader):
    model.eval()
    test_preds = []
    test_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            if images.size(0) == 0:
                continue
            
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()
                model.cuda()
            
            outputs = model(images)
            preds = (outputs > 0.5).float()
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_labels, test_preds)
    print(f'Test Accuracy: {accuracy:.4f}')

train_model(model, train_loader, criterion, optimizer, epochs=70)

validate_model(model, valid_loader)

test_model(model, test_loader)

if train_dataset.missing_files:
    print(f"Toplam eksik dosya sayısı: {len(train_dataset.missing_files)}")
    print("Eksik dosyalar:")
    for missing_file in train_dataset.missing_files:
        print(missing_file)

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import exposure
import pandas as pd


csv_file_path = 'C:/Users/BUSRA/Documents/Teknofest-2024/Dataframe_NPY.csv'  # Kendi CSV dosyanızın yolunu belirtin


df = pd.read_csv(csv_file_path)


class MammographyDataset(Dataset):
    def __init__(self, dataframe, transform=None, apply_clahe=False):
        self.dataframe = dataframe
        self.transform = transform
        self.apply_clahe = apply_clahe
        self.missing_files = []

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['NPY DOSYA YOLU']
        birads_label = self.dataframe.iloc[idx]['birads']
        etiket_adi = self.dataframe.iloc[idx]['ETİKET ADI']

     
        birads_label = 1 if birads_label >= 2 else 0
        
       
        etiket_mapping = {'Kitle': 0, 'Kalsifikasyon': 1, 'Normal': 2}
        etiket_label = etiket_mapping[etiket_adi]

        try:
            img_data = np.load(img_path)
            img_data = Image.fromarray(img_data)
            
            if self.apply_clahe:
                img_data = np.array(img_data)
                img_data = exposure.equalize_adapthist(img_data)  # CLAHE uygulama
                img_data = Image.fromarray((img_data * 255).astype(np.uint8))

            if img_data.mode != 'L':
                img_data = img_data.convert('L')

            if self.transform:
                img_data = self.transform(img_data)

            return img_data, (birads_label, etiket_label)

        except FileNotFoundError:
            self.missing_files.append(img_path)
            print(f"Dosya bulunamadı: {img_path}")
            return None, (None, None)

def custom_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    
    if len(batch) == 0:
        return torch.empty(0), (torch.empty(0), torch.empty(0))
    
    images, labels = zip(*batch)
    birads_labels, etiket_labels = zip(*labels)
    return torch.stack(images, 0), (torch.tensor(birads_labels), torch.tensor(etiket_labels))

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=3),  # ResNet ve AlexNet RGB bekler
    transforms.ToTensor()
])

train_dataset = MammographyDataset(dataframe=train_df, transform=transform, apply_clahe=True)
valid_dataset = MammographyDataset(dataframe=valid_df, transform=transform)
test_dataset = MammographyDataset(dataframe=test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)


class EnsembleModel(nn.Module):
    def __init__(self):
        super(EnsembleModel, self).__init__()
        
        # ResNet50 modeli
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Son sınıflandırma katmanını kaldır
        
        # AlexNet modeli
        self.alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
        self.alexnet.classifier[6] = nn.Identity()  # Son sınıflandırma katmanını kaldır
        
        # DNN katmanı, doğru çıktı boyutlarına göre ayarlandı
        self.dnn_birads = nn.Sequential(
            nn.Linear(2048 + 4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),  # Binary classification output for birads
            nn.Sigmoid()
        )

        self.dnn_etiket = nn.Sequential(
            nn.Linear(2048 + 4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 3), 
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x1 = self.resnet(x)  # ResNet50 çıktısı
        x2 = self.alexnet(x)  # AlexNet çıktısı

        # Her iki modelden gelen özellikleri birleştir
        x_combined = torch.cat((x1, x2), dim=1)

        # Ayrı çıkışlar
        birads_out = self.dnn_birads(x_combined)
        etiket_out = self.dnn_etiket(x_combined)
        return birads_out, etiket_out


def load_checkpoint(filepath, model, optimizer):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch


def save_checkpoint(epoch, model, optimizer, filename='checkpoint.pth.tar'):
    state = {'epoch': epoch,
             'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict()}
    torch.save(state, filename)

model = EnsembleModel()
criterion_birads = nn.BCELoss()
criterion_etiket = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


start_epoch = 0
checkpoint_path = 'checkpoint.pth.tar'
if os.path.exists(checkpoint_path):
    model, optimizer, start_epoch = load_checkpoint(checkpoint_path, model, optimizer)
    print(f"Checkpoint yükleniyor - Eğitim {start_epoch}. epoch'tan devam edecek.")


def train_model(model, train_loader, criterion_birads, criterion_etiket, optimizer, epochs=20, start_epoch=0):
    model.train()
    for epoch in range(start_epoch, epochs):
        running_loss_birads = 0.0
        running_loss_etiket = 0.0
        all_preds_birads = []
        all_labels_birads = []
        all_preds_etiket = []
        all_labels_etiket = []
        
        for images, (birads_labels, etiket_labels) in train_loader:
            if images.size(0) == 0:
                continue
            
           
            if torch.cuda.is_available():
                images, birads_labels, etiket_labels = images.cuda(), birads_labels.cuda(), etiket_labels.cuda()
                model.cuda()
            
            optimizer.zero_grad()
            
            try:
                outputs_birads, outputs_etiket = model(images)
            except RuntimeError as e:
                print(f"Model forward geçiş hatası: {e}")
                print(f"Girdi boyutları: {images.shape}")
                continue

            loss_birads = criterion_birads(outputs_birads, birads_labels.unsqueeze(1).float())
            loss_etiket = criterion_etiket(outputs_etiket, etiket_labels)
            total_loss = loss_birads + loss_etiket
            
            try:
                total_loss.backward()
            except RuntimeError as e:
                print(f"Loss backward geçiş hatası: {e}")
                continue

            optimizer.step()
            running_loss_birads += loss_birads.item()
            running_loss_etiket += loss_etiket.item()

        
            all_preds_birads.extend((outputs_birads > 0.5).cpu().numpy())
            all_labels_birads.extend(birads_labels.cpu().numpy())
            
            _, preds_etiket = torch.max(outputs_etiket, 1)
            all_preds_etiket.extend(preds_etiket.cpu().numpy())
            all_labels_etiket.extend(etiket_labels.cpu().numpy())

        accuracy_birads = accuracy_score(all_labels_birads, all_preds_birads)
        accuracy_etiket = accuracy_score(all_labels_etiket, all_preds_etiket)
        print(f'Epoch {epoch+1}/{epochs}, Loss Birads: {running_loss_birads/len(train_loader)}, Loss Etiket: {running_loss_etiket/len(train_loader)}, Accuracy Birads: {accuracy_birads:.4f}, Accuracy Etiket: {accuracy_etiket:.4f}')
        
      
        save_checkpoint(epoch+1, model, optimizer, filename=checkpoint_path)

# Doğrulama Fonksiyonu
def validate_model(model, valid_loader):
    model.eval()
    val_preds_birads = []
    val_labels_birads = []
    val_preds_etiket = []
    val_labels_etiket = []

    with torch.no_grad():
        for images, (birads_labels, etiket_labels) in valid_loader:
            if images.size(0) == 0:
                continue
            
           
            if torch.cuda.is_available():
                images, birads_labels, etiket_labels = images.cuda(), birads_labels.cuda(), etiket_labels.cuda()
                model.cuda()
            
            outputs_birads, outputs_etiket = model(images)
            
            val_preds_birads.extend((outputs_birads > 0.5).cpu().numpy())
            val_labels_birads.extend(birads_labels.cpu().numpy())
            
            _, preds_etiket = torch.max(outputs_etiket, 1)
            val_preds_etiket.extend(preds_etiket.cpu().numpy())
            val_labels_etiket.extend(etiket_labels.cpu().numpy())

    accuracy_birads = accuracy_score(val_labels_birads, val_preds_birads)
    accuracy_etiket = accuracy_score(val_labels_etiket, val_preds_etiket)
    print(f'Validation Accuracy Birads: {accuracy_birads:.4f}, Validation Accuracy Etiket: {accuracy_etiket:.4f}')

# Test Fonksiyonu
def test_model(model, test_loader):
    model.eval()
    test_preds_birads = []
    test_labels_birads = []
    test_preds_etiket = []
    test_labels_etiket = []

    with torch.no_grad():
        for images, (birads_labels, etiket_labels) in test_loader:
            if images.size(0) == 0:
                continue
            
            if torch.cuda.is_available():
                images, birads_labels, etiket_labels = images.cuda(), birads_labels.cuda(), etiket_labels.cuda()
                model.cuda()
            
            outputs_birads, outputs_etiket = model(images)
            
            test_preds_birads.extend((outputs_birads > 0.5).cpu().numpy())
            test_labels_birads.extend(birads_labels.cpu().numpy())
            
            _, preds_etiket = torch.max(outputs_etiket, 1)
            test_preds_etiket.extend(preds_etiket.cpu().numpy())
            test_labels_etiket.extend(etiket_labels.cpu().numpy())

    accuracy_birads = accuracy_score(test_labels_birads, test_preds_birads)
    accuracy_etiket = accuracy_score(test_labels_etiket, test_preds_etiket)
    print(f'Test Accuracy Birads: {accuracy_birads:.4f}, Test Accuracy Etiket: {accuracy_etiket:.4f}')

train_model(model, train_loader, criterion_birads, criterion_etiket, optimizer, epochs=20, start_epoch=start_epoch)


validate_model(model, valid_loader)


test_model(model, test_loader)


torch.save(model.state_dict(), 'ensemble_model.pth')


if train_dataset.missing_files:
    print(f"Toplam eksik dosya sayısı: {len(train_dataset.missing_files)}")
    print("Eksik dosyalar:")
    for missing_file in train_dataset.missing_files:
        print(missing_file)

train_model(model, train_loader, criterion_birads, criterion_etiket, optimizer, epochs=20, start_epoch=start_epoch)


In [54]:
import torch
import torch.nn as nn
from torchvision import models
import json

class EnsembleModel(nn.Module):
    def __init__(self):
        super(EnsembleModel, self).__init__()
        
        # ResNet50 modeli
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity() 
        
        # AlexNet modeli
        self.alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
        self.alexnet.classifier[6] = nn.Identity() 
        
       
        self.dnn_birads = nn.Sequential(
            nn.Linear(2048 + 4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1), 
            nn.Sigmoid()
        )

        self.dnn_etiket = nn.Sequential(
            nn.Linear(2048 + 4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 3),  
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x1 = self.resnet(x)  
        x2 = self.alexnet(x)  

     
        x_combined = torch.cat((x1, x2), dim=1)

        # Ayrı çıkışlar
        birads_out = self.dnn_birads(x_combined)
        etiket_out = self.dnn_etiket(x_combined)
        return birads_out, etiket_out


def load_checkpoint(filepath, model):
    checkpoint = torch.load(filepath, weights_only=True) 
    model.load_state_dict(checkpoint['model_state_dict'])
    return model


model = EnsembleModel()
model = load_checkpoint(checkpoint_path, model)
model.eval()  


if torch.cuda.is_available():
    model.cuda()


def predict(model, test_loader):
    model.eval()
    predictions = {}

    with torch.no_grad(): 
        for images, image_names in test_loader:
            if torch.cuda.is_available():
                images = images.cuda()

            outputs_birads, outputs_etiket = model(images)
            
        
            for i, name in enumerate(image_names):
                birads_confidence = outputs_birads[i].item()
                etiket_label = torch.argmax(outputs_etiket[i]).item()
                etiket_confidence = outputs_etiket[i].max().item()
                
                predictions[name] = {
                    "birads_confidence": birads_confidence,
                    "etiket_label": etiket_label,
                    "etiket_confidence": etiket_confidence
                }

    return predictions


def save_predictions_to_json(predictions, output_file='tahminler.json'):
    output_data = {
        "kunye": {
            "takim_adi": "DIAGNOS-AI",  # Takım adı
            "takim_id": "344193",        # Takım ID
            "aciklama": "Model tahmin sonuçları",
            "versiyon": "1.0"
        },
        "tahminler": {}
    }

    for name, pred in predictions.items():
        output_data["tahminler"][name] = {
            "kategori": str(pred["etiket_label"]),
            "dosya": {
                name: {
                    "kitle": [
                        {"kutu": "0,0;1,1;1,0;0,1", "guven": str(pred["birads_confidence"])}  # Örnek kutu değerleri
                    ],
                    "kalsifikasyon": []  
                }
            }
        }

    with open(output_file, 'w') as json_file:
        json.dump(output_data, json_file, indent=4)





In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os


# Dataset sınıfı tanımı


# Görüntülerin bulunduğu ana dizin
root_dir = 'C:/Users/BUSRA/Documents/Teknofest-2024/FİNAL'



# Görselleri modelden geçirme
for idx in range(len(test_dataset)):
    image, image_path = test_dataset[idx] 
    
    if torch.cuda.is_available():
        image = image.cuda()

    outputs_birads, outputs_etiket = model(image.unsqueeze(0)) 
    
    print(f"Görüntü: {image_path}")
    print("Birads Output:", outputs_birads)        
    print("Etiket Output:", outputs_etiket)         
    break 


# BIRADS ve Etiket tahminlerini işlemeyi gösteren örnek
for images, image_names in test_loader:
    if torch.cuda.is_available():
        images = images.cuda()

    # Modelden tahminleri al
    outputs_birads, outputs_etiket = model(images)

    birads_pred = (outputs_birads > 0.5).float()  # 0.5 üstü pozitif, altı negatif
    print("BIRADS Tahminleri:", birads_pred)

    # Etiketler için en yüksek olasılığı bulma
    etiket_pred = torch.argmax(outputs_etiket, dim=1)  
    print("Etiket Tahminleri:", etiket_pred)

    break  # Sadece bir batch test etmek için


In [ ]:

for images, image_names in test_loader:
    if torch.cuda.is_available():
        images = images.cuda()

   
    outputs_birads, outputs_etiket = model(images)

    # BIRADS için 0.5 eşik değeri kullanarak sınıflandırma
    birads_pred = (outputs_birads > 0.5).float()  # 0.5 üstü pozitif, altı negatif
    print("BIRADS Tahminleri:", birads_pred)

    # Etiketler için en yüksek olasılığı bulma
    etiket_pred = torch.argmax(outputs_etiket, dim=1)  # En yüksek olasılık hangi sınıfta?
    print("Etiket Tahminleri:", etiket_pred)

    break  # Sadece bir batch test etmek için


In [15]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("GPU Device Available:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.17.0
GPU Device Available: []


In [24]:
import os
csv_file = 'C:/Users/BUSRA/Documents/Teknofest-2024/Dataframe_NPY.csv'

if os.path.exists(csv_file):
    print("Dosya bulundu!")
else:
    print("Dosya bulunamadı. Lütfen dosya yolunu kontrol edin.")


Dosya bulundu!


In [ ]:
import torch
print("PyTorch versiyonu:", torch.__version__)


In [ ]:
import tensorflow as tf
print("TensorFlow versiyonu:", tf.__version__)
